# 5pDEER DEMO
 The aim of this script is to set out the basic design/implemntaion for running incoherent pump pulses on the C Floor AWG.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autoDeer.hardware.keysight_awg as ksawg
import autoDeer.hardware.pulses as pulses

### Connecting to the AWG
The Sequencing mode on both channels should be "STSC"

In [2]:
awg = ksawg.Interface()
awg.open_con('129.132.218.87')
awg._inst_init()
print("Checking Sequencing mode:")
awg.getFunctionMode(3)
sampling_freq = 12e9
sampling_period = 1/sampling_freq
grad = 64


Socket Created
Identification String: b'Agilent Technologies,M8190A,DE52200268,5.0.14.0-2\n'
Checking Sequencing mode:


### Building the Waveforms

In [9]:
tpump = 100
nu = [1.6+0.07,1.6+0.07+0.15]
[p_real,p_imag] = pulses.HSorder1(12,
                                tpump,
                                nu,
                                HSbeta=10,
                                HSorder=[1,6],
                                phase=0,
                                scale=1)
wave1 = ksawg.Waveform(awg,p_real)
wave1.pulse_blanking()
wave1.enforce_gradualrity()
wave1.set_amp('max')
wave1._awg_check()
# wave1.plot().show()
wave2 = ksawg.Waveform(awg,p_imag)
wave2.pulse_blanking()
wave2.enforce_gradualrity()
wave2.set_amp('max')
wave2._awg_check()
# wave2.plot().show()

[p_real,p_imag] = pulses.HSorder1(12,
                                tpump,
                                np.flip(nu),
                                HSbeta=10,
                                HSorder=np.flip([1,6]),
                                phase=0,
                                scale=1)
wave3 = ksawg.Waveform(awg,p_real)
wave3.pulse_blanking()
wave3.enforce_gradualrity()
wave3.set_amp('max')
wave3._awg_check()
# wave3.plot().show()
wave4 = ksawg.Waveform(awg,p_imag)
wave4.pulse_blanking()
wave4.enforce_gradualrity()
wave4.set_amp('max')
wave4._awg_check()
# wave4.plot().show()


True

#### Sending Waveform to AWG

In [8]:
awg.Abort(3)
awg.deleteAllWaveforms(3)
awg.defineWaveform(1,3008,0,ch=3)
awg.defineWaveform(2,3008,0,ch=3)
awg.setWaveform(wave1,1,ch=1)
awg.setWaveform(wave2,2,ch=1)
awg.setWaveform(wave3,1,ch=2)
awg.setWaveform(wave4,2,ch=2)

### Buidling the Sequence Table

In [6]:
seq1 = ksawg.Sequence(awg)
seq1.encode(
    {"data_cmd":0,
"init_marker_seq":1,
"marker_enable":1,
"seg_adv_mode":3,
"seq_loop_cnt":1,
"seg_loop_cnt":1,
"seg_id":1,
"seg_start_offset":0,
"seg_end_offset":3007}
 )
seq2 = ksawg.Sequence(awg)
seq2.encode(
    {"data_cmd":0,
"init_marker_seq":0,
"end_marker_seq":1,
"end_marker_sce":1,
"marker_enable":1,
"seg_adv_mode":3,
"seq_loop_cnt":1,
"seg_loop_cnt":1,
"seg_id":2,
"seg_start_offset":0,
"seg_end_offset":3007}
 )


285409280
1627586560


'1627586560,1,1,2,0,3007'

In [7]:
seq_table = ksawg.SequenceTable(awg)
seq_table.table = [seq1,seq2]
seq_table.write_to_str()

'285409280,1,1,1,0,3007,1627586560,1,1,2,0,3007'

In [10]:
awg.resetSequenceTable()
awg.setSequenceTable(seq_table,0,3)